In [1]:
import copy
from dataclasses import dataclass
from typing import List, Optional

In [2]:
@dataclass
class Job:
    index: int
    p: int
    C: Optional[int] = None

    def __repr__(self) -> str:
        return f'J({self.index})'

    def copy(self):
        return copy.copy(self)


@dataclass
class InstanceSeq:
    d: int
    jobs: List[Job]


class Sequence:
    def __init__(self) -> None:
        self.J1 = []
        self.J2 = []

    @property
    def jobs(self) -> List[Job]:
        return self.J1 + self.J2

    def append_front(self, job: Job):
        self.J1.append(job)

    def append_back(self, job: Job):
        self.J2.insert(0, job)

    def best_insert_front(self, job: Job):
        if len(self.J1) == 0:
            self.J1 = [job]
            return
        for i in range(len(self.J1)):
            if self.J1[i].p <= job.p:
                self.J1.insert(i, job)
                return
        self.J1.append(job)

    def best_insert_back(self, job: Job):
        if len(self.J2) == 0:
            self.J2 = [job]
            return
        for i in range(len(self.J2)):
            if self.J2[i].p >= job.p:
                self.J2.insert(i, job)
                return
        self.J2.append(job)

    def __repr__(self) -> str:
        return str(self.jobs)


@dataclass
class Solution:
    cost: int
    seq: Sequence

In [3]:
def comp_cost(instance: InstanceSeq, sol: Sequence):
    cost = 0
    C = 0
    for j in sol.jobs:
        C += j.p
        j.C = C
        if C <= instance.d:
            cost += instance.d - C
        else:
            cost += C - instance.d
    return Solution(cost=cost, seq=sol)


def heuristic(instance: InstanceSeq) -> Solution:
    queue = instance.jobs.copy()
    queue.sort(key=lambda x: x.p, reverse=True)
    sol = Sequence()
    tao1 = instance.d
    tao2 = sum(j.p for j in queue) - instance.d
    while len(queue) > 0:
        job = queue.pop(0)
        if tao1 >= tao2:
            sol.append_front(job)
            tao1 -= job.p
        else:
            sol.append_back(job)
            tao2 -= job.p
    return comp_cost(instance, sol)

In [4]:
p = [18, 16, 16, 14, 12, 11, 11, 8, 7, 5, 3, 2]
jobs = [Job(i, pi) for i, pi in enumerate(p)]
instance = InstanceSeq(d=42, jobs=jobs)

In [5]:
sol = heuristic(instance)
print(sol.cost)
print(sol.seq)
print([j.p for j in sol.seq.jobs])
print([j.C for j in sol.seq.jobs])
print([max(0, j.C - instance.d, instance.d - j.C) for j in sol.seq.jobs])
print(sum(max(0, j.C - instance.d, instance.d - j.C) for j in sol.seq.jobs))

308
[J(3), J(5), J(7), J(8), J(11), J(10), J(9), J(6), J(4), J(2), J(1), J(0)]
[14, 11, 8, 7, 2, 3, 5, 11, 12, 16, 16, 18]
[14, 25, 33, 40, 42, 45, 50, 61, 73, 89, 105, 123]
[28, 17, 9, 2, 0, 3, 8, 19, 31, 47, 63, 81]
308


In [6]:
def move10(instance: InstanceSeq, sol: Solution, k1):
    seq_alt = copy.deepcopy(sol.seq)
    j1 = seq_alt.J1.pop(k1)
    seq_alt.best_insert_back(j1)
    return comp_cost(instance, seq_alt)


def move01(instance: InstanceSeq, sol: Solution, k2):
    seq_alt = copy.deepcopy(sol.seq)
    j2 = seq_alt.J2.pop(k2)
    seq_alt.best_insert_front(j2)
    return comp_cost(instance, seq_alt)


def move11(instance: InstanceSeq, sol: Solution, k1, k2):
    seq_alt = copy.deepcopy(sol.seq)
    j1 = seq_alt.J1.pop(k1)
    j2 = seq_alt.J2.pop(k2)
    seq_alt.best_insert_back(j1)
    seq_alt.best_insert_front(j2)
    return comp_cost(instance, seq_alt)


def best_improvement(instance: InstanceSeq, sol: Solution):
    best_sol = sol
    for k1 in range(len(sol.seq.J1)):
        sol_alt = move10(instance, sol, k1)
        if sol_alt.cost < best_sol.cost:
            best_sol = sol_alt
    for k2 in range(len(sol.seq.J2)):
        sol_alt = move01(instance, sol, k2)
        if sol_alt.cost < best_sol.cost:
            best_sol = sol_alt
    for k1 in range(len(sol.seq.J1)):
        for k2 in range(len(sol.seq.J2)):
            sol_alt = move11(instance, sol, k1, k2)
            if sol_alt.cost < best_sol.cost:
                best_sol = sol_alt
    return best_sol

In [7]:
best_sol = best_improvement(instance, sol)
print(best_sol)

Solution(cost=308, seq=[J(3), J(5), J(7), J(8), J(11), J(10), J(9), J(6), J(4), J(2), J(1), J(0)])


In [8]:
from pydantic import ValidationError

In [10]:
issubclass(ValidationError, ValueError)

True